In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import random

# Fit language model on Wordpress data

In [4]:
bs, bptt = 64, 70

In [3]:
#python -m spacy download fr

my_tok = spacy.load('fr')

def my_spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

In [4]:
PATH = 'data/txt/wordpress/'
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
md = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=10)

In [5]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(123, 4002, 1, 558973)

In [6]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [7]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip = 0.3

In [9]:
learner.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss   
    0      5.41836    5.1515    



In [10]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      4.964747   4.861962  
    1      4.710859   4.511409  
    2      4.499228   4.398448  
    3      4.389906   4.205618  
    4      4.214793   4.045292  
    5      4.100974   3.948904  
    6      4.032428   3.925189  
    7      4.065681   3.863705  
    8      3.962731   3.748533  
    9      3.858471   3.633356  
    10     3.764991   3.536353  
    11     3.685439   3.447273  
    12     3.616239   3.393011  
    13     3.578113   3.365949  
    14     3.576423   3.36475   



In [11]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      3.621457   3.424829  
    1      3.614206   3.343866  
    2      3.550171   3.273521  
    3      3.47219    3.167678  
    4      3.415134   3.09017   
    5      3.342923   3.012518  
    6      3.283233   2.959568  
    7      3.243617   2.922631  
    8      3.208909   2.923379  
    9      3.20387    2.886638  



In [12]:
learner.fit(1e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      3.254793   2.883765  
    1      3.270128   2.881448  
    2      3.239029   2.847525  
    3      3.232687   2.858032  
    4      3.206249   2.821148  
    5      3.171746   2.772858  
    6      3.147078   2.755853  
    7      3.194739   2.779003  
    8      3.170617   2.7595    
    9      3.157458   2.715929  
    10     3.121693   2.688937  
    11     3.096725   2.668138  
    12     3.077755   2.633148  
    13     3.077113   2.623319  
    14     3.077111   2.6103    



In [13]:
learner.fit(1e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      3.110921   2.705874  
    1      3.108543   2.645146  
    2      3.063471   2.617466  
    3      3.111695   2.619941  
    4      3.034176   2.557742  
    5      3.015316   2.519851  
    6      2.985358   2.496241  
    7      2.965874   2.478436  
    8      2.943421   2.474756  
    9      2.976183   2.489776  



# Show most common predictions

In [14]:
m = learner.model.cpu()

In [15]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [16]:
s = "j'aime"
t = num_str(s)

In [17]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [18]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['le', 'la', 'les', 'l’', 'se', 'vous', 'voyager', 'faire', 'pouvoir', 'leur']

# Sample the model

In [116]:
def sample_model(m, s, l=100):
    
    result = ''
    
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    result += s + ' '

    for i in range(l):
        n = res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        result += word + ' '
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0).unsqueeze(0))

    m[0].bs=bs
    
    return result

In [109]:
m = learner.model.cpu()

In [117]:
sample_model(m, 'je suis')

'je suis obligé de me faire plaisir . je ne sais pas si ce voyage est une ville très touristique , très grande , c’ est aussi la grande ville qui m’ a fait pour se faire inviter . je suis bien content de me voir seul et de me faire inviter à dormir chez l’ habitant . je me suis aussi fait un peu de temps pour moi , et pour un autre voyage en australie . je n’ ai pas eu de problème à me faire plaisir . je ne sais pas si ce voyage est une expérience très touristique '

# Finetune on blogger data

In [24]:
PATH = 'data/txt/blogger'
md2 = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=10)

In [25]:
len(md2.trn_dl), md2.nt, len(md2.trn_ds), len(md2.trn_ds[0].text)

(20, 4002, 1, 95914)

In [82]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [83]:
learner2 = md2.get_model(opt_fn, em_sz, nh, nl, dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
learner2.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner2.clip = 0.3

In [29]:
learner2.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss   
    0      4.07943    3.612095  



In [30]:
learner2.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      3.729076   3.4321    
    1      3.649061   3.236375  
    2      3.53968    3.166946  
    3      3.483248   3.033893  
    4      3.402295   2.859933  
    5      3.293501   2.742957  
    6      3.212594   2.733846  
    7      3.17222    2.634837  
    8      3.112819   2.48527   
    9      3.032762   2.337774  
    10     2.938747   2.240985  
    11     2.830832   2.172124  
    12     2.736191   2.025528  
    13     2.659064   1.991474  
    14     2.600195   1.999561  



In [31]:
learner2.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      2.508398   2.028785  
    1      2.506911   1.910381  
    2      2.477282   1.858803  
    3      2.442942   1.696492  
    4      2.39218    1.615291  
    5      2.311126   1.52568   
    6      2.239762   1.448574  
    7      2.192392   1.445718  
    8      2.15074    1.409442  
    9      2.128195   1.430044  



In [32]:
learner2.fit(1e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      2.045861   1.391264  
    1      2.07245    1.361346  
    2      2.109661   1.374343  
    3      2.065975   1.356039  
    4      2.026473   1.310358  
    5      2.025779   1.267039  
    6      1.988473   1.310442  
    7      2.001189   1.421678  
    8      1.987376   1.272208  
    9      1.94994    1.184513  
    10     1.92364    1.159139  
    11     1.913576   1.145634  
    12     1.931912   1.139126  
    13     1.928751   1.177401  
    14     1.901622   1.257719  



In [33]:
learner2.fit(1e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss   
    0      1.806324   1.147617  
    1      1.825615   1.086717  
    2      1.792029   1.049193  
    3      1.825028   1.042164  
    4      1.783125   1.027698  
    5      1.786378   0.990423  
    6      1.781774   1.047837  
    7      1.764112   1.0085    
    8      1.72583    1.004652  
    9      1.732903   0.980324  



In [57]:
learner2.fit(3e-3, 6, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   
    0      1.847859   1.060709  
    1      1.813359   1.026883  
    2      1.778089   0.948474  
    3      1.824945   1.080708  
    4      1.800656   0.993749  
    5      1.737481   0.902956  
    6      1.709092   0.892642  
    7      1.695567   0.864817  
    8      1.700057   0.84268   
    9      1.672589   0.770739  
    10     1.623281   0.716308  
    11     1.584262   0.687014  
    12     1.523147   0.624581  
    13     1.483324   0.658361  
    14     1.474085   0.621477  
    15     1.520616   0.714385  
    16     1.520002   0.642848  
    17     1.515244   0.60481   
    18     1.472622   0.54816   
    19     1.449795   0.688151  
    20     1.420494   0.504746  
    21     1.417902   0.512269  
    22     1.363959   0.54308   
    23     1.324987   0.447386  
    24     1.280399   0.387428  
    25     1.230288   0.366208  
    26     1.175008   0.361138  
    27     1.143298   0.340434  
    28     1.104044   0.444046  
    29   

# Sample the model

In [121]:
m2 = learner2.model.cpu()

In [124]:
sample_model(m2, 'je suis')

'je suis fatigué du stop qui ne marche pas , et je dois marcher pour la nuit . de toute façon , je vais en profiter pour faire un petit tour de terrain de foot très haut , mais vu que le paysage est en train de dormir dans le désert . je sais bien que le terrain est à peine pour sortir de la paz , mais je suis dans le vent . la douleur se termine plutôt . je suis un peu trop faible pour que ça soit normal , et je me fais embarquer en chemin par un groupe '

# Generate sample article

In [86]:
txt = sample_model(m2, 'je suis', 100000)

In [1]:
txt = open('data/sampled.txt').read()

In [2]:
# Handle extra spaces
txt = txt.replace('\xa0', ' ')  # non-breaking spaces
txt = txt.replace(' ,', ',')
txt = txt.replace(' ,', ',')
txt = txt.replace(' :', ':')
txt = txt.replace(' ;', ';')
txt = txt.replace(' / ', '/')
txt = txt.replace("' ", "'")
txt = txt.replace('( ', '(')
txt = txt.replace(' )', ')')
for _ in range(3):
    txt = txt.replace('. .', '..')
    txt = txt.replace('  ', ' ')

In [3]:
# Capitalize letters after '.'
txt = '. '.join(s.strip().capitalize() for s in txt.split('.'))

In [4]:
# Strip text so that it ends with '.'
last_full_stop_idx = txt.rfind('.')
txt = txt[:last_full_stop_idx + 1]

In [5]:
# Split into paragraphs
N_SENTENCES = 20
sentences = [s.strip() for s in txt.split('.')]
txt = '\n\n'.join('. '.join(sentences[i:i + N_SENTENCES]) + '.' for i in range(0, len(sentences), N_SENTENCES))
txt = txt.replace('. .', '.')

In [6]:
print(txt[:2000])

Je suis fatigué du stop qui ne marche pas, et je dois marcher pour la nuit. De toute façon, je vais en profiter pour faire un petit tour de terrain de foot très haut, mais vu que le paysage est en train de dormir dans le désert. Je sais bien que le terrain est à peine pour sortir de la paz, mais je suis dans le vent. La douleur se termine plutôt. Je suis un peu trop faible pour que ça soit normal, et je me fais embarquer en chemin par un groupe de français qui y va en pente. Je ne peux pas dire que sa mort me touche pas, après avoir lu tant de ses amis. Elle est désormais dans le pays, mais pourtant, ça sera à la douane. Bon, maintenant, les choses ne se termine jamais comme prévu, et à un moment, il est possible que les moustiques sont très sympathiques. Ils sont en train de faire du stop tout juste la douane pour ne plus faire par la fenêtre. En fait, elle ne se termine pas, et je les fais à un moment, et je demande au bout de 500 mètres. Je ne compte que trois de leurs enfants sur c

In [7]:
print(txt[-2000:])

soleil ne marche, et je me fais embarquer par un bus. D'ailleurs, il est temps de faire sécher les vêtements et les vêtements dans les jambes. Je remercie mes conducteurs qui en passent. Je ne comprends pas trop où sont passés les artistes de rue argentins si présents à bord. Il y a bien un moment où il faut retourner à la maison, et vu que je suis malade. Je me fais embarquer par 2 taxis de suite (même si ça peut fait 3 jours, il est temps de mettre un peu de temps pour traverser de la colombie. Et, pour les deux raisons que voici, ce ne sont pas des vacances, merci. Le lendemain, je retrouve mes affaires sur la route, et je vais voir le conducteur du bus. Le conducteur me demande de venir voir si je fais un signe de tour. Il peut se rendre finalement, vu que je ne suis pas un problème de route. Je le remercie, et je lui demande si il a une idée d'où je cherche un endroit pour planter la tente. D'ailleurs il fait nuit et il va falloir y penser, mais ma bouteille d'eau est vide et je s

In [8]:
with open('data/generated/words_sample.txt', 'w') as f:
    f.write(txt + '\n')